In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


In [2]:
#hide warnings
HTML('''<script>
code_show_err=true; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output errors, click <a href="javascript:code_toggle_err()">here</a>.''')

In [3]:
print('Testing GAL4 lines upstream to 64A11-LexA. Secondary round with images taken from posterior and here comparing responses in regular saline (containing glucose and trehalose) with starvation saline (containing no glucose or trehalose).')
print('Project A83, Daniel Bushey')


Testing GAL4 lines upstream to 64A11-LexA. Secondary round with images taken from posterior and here comparing responses in regular saline (containing glucose and trehalose) with starvation saline (containing no glucose or trehalose).
Project A83, Daniel Bushey


In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [8]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import ccModules as cc
import ccModules2 as cc2

from IPython.html.widgets import interact
import pandas as pd
from IPython.display import IFrame
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_file, show



In [9]:
#load project  specific parameters
from A83_init20190121 import *

path1 /home/daniel/Desktop/ResearchUbuntuYoga720/A83_A6411/A83_Data3


In [36]:
#load data
#load pandas data frame
exceldata = pd.read_hdf(os.path.join(saveFig, 'Compiled_data.hdf5'), 'data')
#print(exceldata.shape)
#exceldata = exceldata[exceldata['Grade'] == 1]



In [37]:
#restrict this analysis to specific genotypes
restrict = ['SS33917', 'MB310C', 'MB082C']
exceldata = exceldata[exceldata['Genotype'].isin(restrict)]
print('This analysis is restricted to the following genotypes.')
print(exceldata['Genotype'].unique())

This analysis is restricted to the following genotypes.
['MB082C' 'MB310C' 'SS33917']


In [38]:
print('Checking how many sessions included in the loaded data set:')
exceldata = exceldata[~exceldata['Sample Name'].str.contains('A83-26')]
print(len(exceldata))

Checking how many sessions included in the loaded data set:
36


In [39]:
#pull out roi data and add each roi as separate row
exceldata = cc2.pullIntensityData(exceldata)

In [40]:
#check parameters within data
parameters_to_compare = ['Genotype', 'Saline']
print('These are the different conditions tested.')
for cp in parameters_to_compare:
    print('Parameter:', cp)
    print(exceldata[cp].unique())

These are the different conditions tested.
Parameter: Genotype
['MB082C' 'MB310C' 'SS33917']
Parameter: Saline
['Regular' 'Starvation']


In [41]:
#Get counts
print('Number of animals tested.')
groups = exceldata.groupby(['Genotype', 'Saline', 'roi'])
groups['roi'].count()


Number of animals tested.


Genotype  Saline      roi       
MB082C    Regular     Background    9
                      Region1       9
                      Region2       8
          Starvation  Background    3
                      Region1       3
                      Region2       3
MB310C    Regular     Background    8
                      Region1       8
                      Region2       6
          Starvation  Background    1
                      Region1       1
                      Region2       1
SS33917   Regular     Background    9
                      Region1       9
                      Region2       8
          Starvation  Background    6
                      Region1       6
                      Region2       5
Name: roi, dtype: int64

In [42]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import Range1d

In [43]:
output_notebook()

Loading BokehJS ...

In [44]:
def concatenate_cgroup(cgroup):
    if isinstance(cgroup, tuple):
        string1 = cgroup[0]
        for i in cgroup[1:]:
            string1 = string1 + '_' + i
    else:
        string1 = cgroup
    return string1

In [45]:
#create a dictionary holding data
raw_data={}
notroi = 'Background' #will look at all rois except background
for cgroup, frame1 in exceldata.groupby(['Genotype', 'Saline']):
    crois = frame1['roi'].unique()
    crois = [ccroi for ccroi in crois if ccroi != notroi]
    
    for croi in crois:
        groupname = concatenate_cgroup(cgroup) + '_' + croi
        frame2 = frame1[frame1['roi'] == croi]
        if len(frame2) != 0:
            frame3 = cc2.intensityDataFrame(frame2)
            raw_data[groupname] = frame3.subtractBackground(frame1)

In [46]:
print('Stimulation Protocol')
volt = cc2.intensityDataFrame(exceldata).getVoltage()
timevolt = np.arange(0, len(volt))/100
plot=figure(y_range = (timevolt[0], timevolt[-1]), x_range = (0, 0.9), plot_width=600, plot_height=200)
plot=figure( plot_width=600, plot_height=200)
source = ColumnDataSource(data=dict(y=volt, x= timevolt ))
plot.line('x', 'y', source = source, line_width = 3, line_color ='red')
plot.x_range = Range1d(0, np.max(timevolt)*1.05)
plot.y_range = Range1d(0,np.max(volt)*1.05)
plot.xaxis.axis_label = 'Time (S)'
plot.yaxis.axis_label = 'Volt (V)'
#plot.line(timevolt,volt)
#output_file("StimProtocol.html", title = 'Stimulation Protocol')
show(plot)

Stimulation Protocol


In [47]:
from bokeh.models import LinearAxis, Range1d
from bokeh.palettes import Spectral5
from bokeh.models import Legend

In [53]:
print('Comparing responses in regular and starvation saline.')
print('Titles for each graph indicated GAL4 and ROI where signal was measured.')

notroi = 'Background' #will look at all rois except background
colors = { 'Regular' : (0,255,0), 'Starvation': (0,0,255)}
timeStamp = cc2.intensityDataFrame(exceldata).getTimeStamp()
for cgroup, frame1 in exceldata.groupby(['Genotype']):
    crois = frame1['roi'].unique()
    crois = [ccroi for ccroi in crois if ccroi != notroi]
    for croi in crois:
        p = figure(plot_width=850, plot_height=500, x_range = Range1d(0, np.max(timeStamp)), x_axis_label = 'Time (S)', y_axis_label = 'Intensity - Background')
        p.title.text = cgroup + ' ' + croi
        #plot showing voltage
        p.extra_y_ranges = {"foo": Range1d(start=-0, end=2)}
        p.add_layout(LinearAxis(y_range_name='foo', axis_label = 'Volt (V)'), 'right')
        c = p.line(timevolt, volt, y_range_name = 'foo', line_color = (255,0, 0), line_alpha=0.5, line_width =2)
        legend_it = [('Voltage', [c])]
        cdata = []
        for cgroup2, frame2 in frame1.groupby(['Saline']):
            groupname = cgroup + '_' + cgroup2 + '_' + croi
            #plot showing mean data results
            data = np.vstack(raw_data[groupname]['intensity'])
            if len(cdata) > 0:
                cdata = np.concatenate([cdata, data])
            else:
                cdata = data
            c = p.line(x=timeStamp, y=np.mean(data, axis=0), line_width=2, line_color = colors[cgroup2])
            legend_it.append(('Mean-' + cgroup2, [c]))
            for row in range(0,data.shape[0]):
                c = p.line(x=timeStamp, y=data[row, :], line_width=2, line_color = colors[cgroup2], line_alpha = 0.3) #line_color = colors[cgroup])
                legend_it.append((raw_data[groupname]['Sample Name'].iloc[row], [c]))
        
        legend = Legend(items = legend_it, location=(0, -20))
        legend.click_policy="hide"
        p.add_layout(legend, 'right')
        
        #change y axis to 
        min1 = np.min(cdata)
        max1 = np.max(cdata)
        p.y_range = Range1d(min1, max1)



        #p.x_range = Range1d(0, 450)
        p.y_range = Range1d(min1, max1)
        #output_file("Mean-RawIntensity.html", title = 'Stimulation Protocol')
        #p.patch(timevolt2, volt2, y_range_name = 'foo')
        show(p)

Comparing responses in regular and starvation saline.
Titles for each graph indicated GAL4 and ROI where signal was measured.
